In [1]:
import pandas as pd
import requests
import os
import numpy as np
import math

### Data

Data jsou od 1.7.2013 do 30.6.2018. Měření proběhla ve městě Leeds v 7 lokacích. Vzhledem k tomu, že AllertoBywater má nejméně NaN hodnot, analýzu budeme provádět na této části.

In [2]:
# stáhnout soubory z url adres
with open('data/urls.txt', 'r') as f:
    for url in f.readlines():
        name = url.split('/')[-1]
        with open(f"data/{name.strip()}", 'wb') as ff:
            ff.write(requests.get(url).content)

In [3]:
# získat názvy souborů
files = [file for file in os.listdir('./data') if file.endswith('.csv')]
files.remove('raingauge-dailytotals.csv')

In [4]:
def get_month(arg):
    return {
        'Jan' : 1,
        'Feb' : 2,
        'Mar' : 3,
        'Apr' : 4,
        'May' : 5,
        'Jun' : 6,
        'Jul' : 7,
        'Aug' : 8,
        'Sep' : 9,
        'Oct' : 10,
        'Nov' : 11,
        'Dec' : 12
    }[arg]

# fix časových údajů, protože nejsou konzistentní
def f(row):
    date = row.TIMESTAMP_LessOne
    if not isinstance(date, str):
        return np.nan
    
    if '-' not in date:
        return date
    
    day, month, year = date.split('-')
    month = str(get_month(month))
    
    if len(month) == 1:
        month = f"0{month}" 
    return f"{day}/{month}/20{year}"

In [5]:
# načtení prvního soubodu (je největší)
df = pd.read_csv('data/raingauge-dailytotals.csv')

In [6]:
# oprava datumů
df.TIMESTAMP_LessOne = df.apply(lambda row : f(row), axis = 1)

In [7]:
# spojení csv souborů do jenoho
for file in files:
    tmp = pd.read_csv(f"data/{file}")
    tmp.TIMESTAMP_LessOne = tmp.apply(lambda row : f(row), axis = 1)
    df = pd.concat([df, tmp])

In [8]:
# drop chybějících datumů
df.dropna(subset=['TIMESTAMP_LessOne'], inplace=True)
df.rename(columns={'TIMESTAMP_LessOne' : 'date'}, inplace=True)

In [9]:
# vzhledem k tomu, že nejméně nan hodnot má AllertonBywater, použijeme tyto data
df.isna().sum()

date                                       0
Qry_AllertonBywater_Daily.RAIN_mm_TOT     61
Qry_Middleton_Daily.RAIN_mm_TOT          244
Qry_Otley_Daily.RAIN_mm_TOT              635
Qry_Shadwell_Daily.RAIN_mm_TOT           157
Qry_Wetherby_Daily.RAIN_mm_TOT           397
Qry_PotteryField_Daily.RAIN_mm_TOT       395
dtype: int64

In [10]:
df.rename(columns={'Qry_AllertonBywater_Daily.RAIN_mm_TOT' : 'rain'}, inplace=True)

In [11]:
# Vzhledem k tomu, že se jedná o jedno město, tak se dá předpokládat, že v průměru prší všude stejně. 
# Proto doplním chybějící hodnoty pomocí ostatních dat.
def f(row):
    if math.isnan(row.rain):
        vals = [num for num in row.values[1:] if not(math.isnan(num))]
        
        return np.mean(vals)
    return row.rain

In [12]:
df['rain'] = df.apply(lambda row : f(row), axis=1)

In [13]:
df.isna().sum()

date                                    0
rain                                    0
Qry_Middleton_Daily.RAIN_mm_TOT       244
Qry_Otley_Daily.RAIN_mm_TOT           635
Qry_Shadwell_Daily.RAIN_mm_TOT        157
Qry_Wetherby_Daily.RAIN_mm_TOT        397
Qry_PotteryField_Daily.RAIN_mm_TOT    395
dtype: int64

In [14]:
# selekce dat, které použijeme
df = df[['date', 'rain']]

In [15]:
# save the csv
df.to_csv('data/raingauges.csv', index=False)